In [1]:
!nvidia-smi

Sat Dec 13 10:40:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   31C    P0             49W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [47]:
import time
import json
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path

import numpy as np
import pandas as pd

try:
    import cudf
    import cupy as cp
    GPU = True
    print(f"✅ cuDF: {cudf.__version__}")
except ImportError:
    import pandas as cudf
    import numpy as cp
    GPU = False
    print("⚠️ CPU mode (pandas)")

✅ cuDF: 25.10.00


In [48]:
# Load Port Activity (main dataset)
print("Loading datasets...")
t0 = time.time()

ports_activity = cudf.read_csv("Daily_Port_Activity_Data_and_Trade_Estimates.csv")
print(f"✅ Port Activity: {len(ports_activity):,} records")

Loading datasets...
✅ Port Activity: 5,010,140 records


In [49]:
# Load Port Database (with lat/lon)
ports_db = cudf.read_csv("PortWatch_ports_database.csv")
print(f"✅ Port Database: {len(ports_db):,} ports")
print(f"   Columns: {list(ports_db.columns)[:10]}...")

✅ Port Database: 1,985 ports
   Columns: ['X', 'Y', 'portid', 'portname', 'country', 'ISO3', 'continent', 'fullname', 'lat', 'lon']...


In [50]:
import json

# Load Rail Nodes
print("Loading Rail Nodes...")
with open("NTAD_North_American_Rail_Network_Nodes_-3731209159413770440.geojson", 'r') as f:
    rail_nodes_raw = json.load(f)

# Parse to dataframe
rail_nodes_list = []
for feat in rail_nodes_raw['features']:
    props = feat['properties']
    coords = feat['geometry']['coordinates']
    rail_nodes_list.append({
        'node_id': props.get('FRANODEID'),
        'country': props.get('COUNTRY'),
        'state': props.get('STATE'),
        'fra_district': props.get('FRADISTRCT'),
        'node_lon': coords[0],
        'node_lat': coords[1]
    })

rail_nodes = pd.DataFrame(rail_nodes_list)
if GPU:
    rail_nodes = cudf.DataFrame(rail_nodes)

print(f"✅ Rail Nodes: {len(rail_nodes):,} nodes")

Loading Rail Nodes...
✅ Rail Nodes: 250,129 nodes


In [51]:
# Load Chokepoints
chokepoints = cudf.read_csv("Daily_Chokepoints_Data.csv")
print(f"✅ Chokepoints: {len(chokepoints):,} records")

✅ Chokepoints: 70,728 records


In [52]:
# Load Freight/Logistics
freight = cudf.read_csv("fFreight.csv")
print(f"✅ Freight: {len(freight):,} records")

✅ Freight: 92,060 records


In [53]:
# Load Disruptions
disruptions = cudf.read_csv("portwatch_disruptions_database_-3602226124776604501.csv")
print(f"✅ Disruptions: {len(disruptions):,} events")

print(f"\n⏱️ All data loaded in {time.time()-t0:.1f}s")

✅ Disruptions: 125 events

⏱️ All data loaded in 4.0s


---
## 2. 🇺🇸 Filter to US Ports

In [54]:
# Filter port activity to US
print("Filtering to US ports...")
ports_us = ports_activity[ports_activity['country'] == 'United States'].copy()
print(f"✅ US Port Activity: {len(ports_us):,} records")

# Filter port database to US
ports_db_us = ports_db[ports_db['country'] == 'United States'].copy()
print(f"✅ US Ports Database: {len(ports_db_us):,} ports")

# Filter rail nodes to US
rail_us = rail_nodes[rail_nodes['country'] == 'US'].copy()
print(f"✅ US Rail Nodes: {len(rail_us):,} nodes")

# Free memory
del ports_activity, ports_db, rail_nodes
import gc; gc.collect()
if GPU:
    cp.get_default_memory_pool().free_all_blocks()

Filtering to US ports...
✅ US Port Activity: 287,736 records
✅ US Ports Database: 114 ports
✅ US Rail Nodes: 197,444 nodes


In [55]:
# Parse dates
print("Parsing dates...")
if GPU:
    ports_us['date'] = ports_us['date'].str.slice(0, 19)
    ports_us['date'] = cudf.to_datetime(ports_us['date'], format='%Y/%m/%d %H:%M:%S')
else:
    ports_us['date'] = pd.to_datetime(ports_us['date'])

print(f"📅 Date range: {ports_us['date'].min()} to {ports_us['date'].max()}")

Parsing dates...
📅 Date range: 2019-01-01T00:00:00.000000000 to 2025-11-28T00:00:00.000000000


---
## 3. 🗺️ Spatial Join: Ports → Rail Terminals

In [56]:
# Get port coordinates from port database
print("Preparing port coordinates...")

# Convert to pandas for spatial operations
ports_db_pd = ports_db_us.to_pandas() if GPU else ports_db_us
rail_pd = rail_us.to_pandas() if GPU else rail_us

# Select relevant columns from port DB
port_coords = ports_db_pd[['portid', 'portname', 'lat', 'lon']].copy()
port_coords = port_coords.rename(columns={'lat': 'port_lat', 'lon': 'port_lon'})

print(f"Ports with coordinates: {len(port_coords)}")
print(port_coords.head())

Preparing port coordinates...
Ports with coordinates: 114
      portid             portname   port_lat   port_lon
32   port767        Morehead City  34.717824 -76.696018
62   port808          New Bedford  41.628425 -70.912106
89   port809            New Haven  41.275747 -72.893150
111  port812          New Orleans  29.930033 -90.130868
128  port815  New York-New Jersey  40.632984 -74.179421


In [58]:
# Haversine distance function
def haversine_distance(lat1, lon1, lat2, lon2):
    """Calculate distance in km between two points."""
    R = 6371  # Earth's radius in km

    lat1_rad = np.radians(lat1)
    lat2_rad = np.radians(lat2)
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)

    a = np.sin(dlat/2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    return R * c

print("✅ Distance function ready")

✅ Distance function ready


In [59]:
# Find nearest rail terminal for each port
print("Spatial join: Finding nearest rail terminal for each port...")
t0 = time.time()

# For each port, find nearest rail node within 100km
MAX_DISTANCE_KM = 100  # Maximum drayage distance

port_terminal_pairs = []

for idx, port in port_coords.iterrows():
    port_lat = port['port_lat']
    port_lon = port['port_lon']

    # Calculate distance to all rail nodes
    distances = haversine_distance(
        port_lat, port_lon,
        rail_pd['node_lat'].values,
        rail_pd['node_lon'].values
    )

    # Find nearest within threshold
    nearest_idx = np.argmin(distances)
    nearest_dist = distances[nearest_idx]

    if nearest_dist <= MAX_DISTANCE_KM:
        nearest_node = rail_pd.iloc[nearest_idx]
        port_terminal_pairs.append({
            'portid': port['portid'],
            'portname': port['portname'],
            'port_lat': port_lat,
            'port_lon': port_lon,
            'terminal_id': nearest_node['node_id'],
            'terminal_state': nearest_node['state'],
            'terminal_lat': nearest_node['node_lat'],
            'terminal_lon': nearest_node['node_lon'],
            'distance_km': nearest_dist
        })

port_terminal_df = pd.DataFrame(port_terminal_pairs)
print(f"\n✅ Spatial join complete in {time.time()-t0:.1f}s")
print(f"   Ports matched to terminals: {len(port_terminal_df)}")
print(f"   Avg distance to terminal: {port_terminal_df['distance_km'].mean():.1f} km")

Spatial join: Finding nearest rail terminal for each port...

✅ Spatial join complete in 1.1s
   Ports matched to terminals: 105
   Avg distance to terminal: 2.8 km


In [60]:
# Preview port-terminal mapping
print("\n🗺️ PORT-TERMINAL MAPPING (sample):")
print(port_terminal_df.head(15).to_string(index=False))


🗺️ PORT-TERMINAL MAPPING (sample):
  portid            portname  port_lat    port_lon  terminal_id terminal_state  terminal_lat  terminal_lon  distance_km
 port767       Morehead City 34.717824  -76.696018       478199             NC     34.718580    -76.696322     0.088574
 port808         New Bedford 41.628425  -70.912106       495226             MA     41.640072    -70.923651     1.611697
 port809           New Haven 41.275747  -72.893150       492181             CT     41.285012    -72.904294     1.388756
 port812         New Orleans 29.930033  -90.130868       396523             LA     29.920936    -90.133003     1.032342
 port815 New York-New Jersey 40.632984  -74.179421       488035             NY     40.632837    -74.179568     0.020472
 port818        Newport News 36.983866  -76.435371       479603             VA     36.986370    -76.429210     0.613984
 port826             Norfolk 36.849627  -76.330492       479936             VA     36.846955    -76.329222     0.317943
 por

---
## 4. 🚚 Add Drayage/Truck Times

In [27]:
# Estimate drayage times based on distance
print("Calculating drayage times...")

# Average truck speed assumptions
AVG_TRUCK_SPEED_KMH = 50  # Average including loading/unloading
LOADING_TIME_MIN = 30     # Container loading time
UNLOADING_TIME_MIN = 30   # Container unloading time

port_terminal_df['drayage_time_min'] = (
    (port_terminal_df['distance_km'] / AVG_TRUCK_SPEED_KMH * 60) +
    LOADING_TIME_MIN + UNLOADING_TIME_MIN
)

# Add variability estimates (P25, P50, P75)
port_terminal_df['drayage_time_p25'] = port_terminal_df['drayage_time_min'] * 0.85
port_terminal_df['drayage_time_p50'] = port_terminal_df['drayage_time_min']
port_terminal_df['drayage_time_p75'] = port_terminal_df['drayage_time_min'] * 1.25

# Estimate cost (based on distance)
COST_PER_KM = 2.50  # USD per km
FIXED_COST = 150    # Base handling fee
port_terminal_df['drayage_cost_usd'] = (
    port_terminal_df['distance_km'] * COST_PER_KM + FIXED_COST
)

print(f"✅ Drayage estimates added")
print(f"   Avg drayage time: {port_terminal_df['drayage_time_min'].mean():.0f} min")
print(f"   Avg drayage cost: ${port_terminal_df['drayage_cost_usd'].mean():.0f}")

Calculating drayage times...
✅ Drayage estimates added
   Avg drayage time: 63 min
   Avg drayage cost: $157


In [33]:
# Calculate truck demand based on container volume
print("\nEstimating truck demand...")

# Truck capacity assumptions
TEU_PER_TRUCK = 2  # Standard: 1 FEU (40ft) = 2 TEU per truck

# Get average import containers per port
ports_us_pd = ports_us.to_pandas() if GPU else ports_us
port_volumes = ports_us_pd.groupby('portid').agg(
    avg_import_teu=('import_container', 'mean'),
    avg_export_teu=('export_container', 'mean'),
    avg_container_calls=('portcalls_container', 'mean')
).reset_index()

# Merge with port-terminal data
port_terminal_df = port_terminal_df.merge(port_volumes, on='portid', how='left')

# Calculate daily truck demand (60% of imports go to rail)
RAIL_SHARE = 0.60
port_terminal_df['daily_trucks_needed'] = (
    port_terminal_df['avg_import_teu'].fillna(0) * RAIL_SHARE / TEU_PER_TRUCK
)

print(f"✅ Truck demand estimated")
print(f"   Total daily trucks needed: {port_terminal_df['daily_trucks_needed'].sum():.0f}")


Estimating truck demand...
✅ Truck demand estimated
   Total daily trucks needed: 57327


---
## 5. 🚨 Surge Detection

In [30]:
# Filter to major ports only (avg > 1 call/day)
print("Detecting surges on major ports...")

# Get port averages
port_avg = ports_us_pd.groupby('portname')['portcalls'].mean()
major_ports = port_avg[port_avg >= 1.0].index.tolist()

print(f"Major ports (≥1 call/day): {len(major_ports)}")

# Filter to major ports
df = ports_us_pd[ports_us_pd['portname'].isin(major_ports)].copy()
df = df.sort_values(['portname', 'date'])

print(f"Records for major ports: {len(df):,}")

Detecting surges on major ports...
Major ports (≥1 call/day): 8
Records for major ports: 20,192


In [31]:
# Calculate rolling statistics
print("Computing rolling statistics...")
t0 = time.time()

df['ma7'] = df.groupby('portname')['portcalls'].transform(
    lambda x: x.rolling(7, min_periods=1).mean()
)
df['ma30'] = df.groupby('portname')['portcalls'].transform(
    lambda x: x.rolling(30, min_periods=1).mean()
)
df['std7'] = df.groupby('portname')['portcalls'].transform(
    lambda x: x.rolling(7, min_periods=1).std()
).fillna(0)

# Z-score
df['zscore'] = (df['portcalls'] - df['ma7']) / df['std7'].replace(0, 1)

# Surge flags
df['surge_2std'] = (df['zscore'] > 2).astype(int)
df['surge_relative'] = (df['portcalls'] > df['ma7'] * 1.5).astype(int)  # 50% above avg

print(f"✅ Rolling stats: {time.time()-t0:.1f}s")

Computing rolling statistics...
✅ Rolling stats: 0.0s


In [32]:
# Surge summary
surge_summary = df.groupby('portname').agg({
    'surge_2std': 'sum',
    'surge_relative': 'sum',
    'portcalls': ['mean', 'max'],
    'zscore': 'max'
}).reset_index()

surge_summary.columns = ['portname', 'surge_2std_days', 'surge_rel_days',
                         'avg_calls', 'max_calls', 'max_zscore']
surge_summary['total_days'] = df.groupby('portname').size().values
surge_summary['surge_rate'] = surge_summary['surge_2std_days'] / surge_summary['total_days'] * 100
surge_summary = surge_summary.sort_values('avg_calls', ascending=False)

print("\n🚨 SURGE ANALYSIS (Major Ports Only):")
print(surge_summary.head(20).to_string(index=False))


🚨 SURGE ANALYSIS (Major Ports Only):
           portname  surge_2std_days  surge_rel_days  avg_calls  max_calls  max_zscore  total_days  surge_rate
        New Orleans               23              98  13.948891         42    2.234217        2524    0.911252
New York-New Jersey               10             109  10.583994         24    2.153094        2524    0.396197
          Baltimore               15             261   4.893819         15    2.244540        2524    0.594295
           Beaumont               16             418   3.284469         12    2.267787        2524    0.633914
            Norfolk               11             312   3.177496          9    2.121320        2524    0.435816
            Oakland               12             358   2.908875          9    2.267787        2524    0.475436
     Wilmington, NC               16             555   1.693740          6    2.267787        2524    0.633914
       Newport News               27             597   1.114501          5

---
## 6. 🔗 Create Unified Freight Model

In [34]:
# Merge port activity with port-terminal mapping
print("Building unified freight model...")

# Add port-terminal info to activity data
unified = df.merge(
    port_terminal_df[['portid', 'portname', 'port_lat', 'port_lon',
                      'terminal_id', 'terminal_state', 'terminal_lat', 'terminal_lon',
                      'distance_km', 'drayage_time_min', 'drayage_cost_usd',
                      'daily_trucks_needed']],
    on=['portid', 'portname'],
    how='left'
)

print(f"✅ Unified dataset: {len(unified):,} records")
print(f"   With terminal mapping: {unified['terminal_id'].notna().sum():,}")

Building unified freight model...
✅ Unified dataset: 20,192 records
   With terminal mapping: 20,192


In [35]:
# Add temporal features
print("Adding temporal features...")

unified['day_of_week'] = pd.to_datetime(unified['date']).dt.dayofweek
unified['is_weekend'] = (unified['day_of_week'] >= 5).astype(int)
unified['month_sin'] = np.sin(2 * np.pi * unified['month'] / 12)
unified['month_cos'] = np.cos(2 * np.pi * unified['month'] / 12)

# Lag features
for lag in [1, 3, 7]:
    unified[f'calls_lag{lag}'] = unified.groupby('portname')['portcalls'].shift(lag)
    unified[f'import_lag{lag}'] = unified.groupby('portname')['import_container'].shift(lag)

# Momentum
unified['momentum_3d'] = unified.groupby('portname')['portcalls'].diff(3)
unified['momentum_7d'] = unified.groupby('portname')['portcalls'].diff(7)

print("✅ Features added")

Adding temporal features...
✅ Features added


In [36]:
# Create targets
print("Creating prediction targets...")

# 24h forecast
unified['target_calls_24h'] = unified.groupby('portname')['portcalls'].shift(-1)
unified['target_surge_24h'] = unified.groupby('portname')['surge_2std'].shift(-1)

# 72h forecast
unified['target_calls_72h'] = unified.groupby('portname')['portcalls'].shift(-3)

# Truck demand forecast (based on import forecast)
unified['target_import_24h'] = unified.groupby('portname')['import_container'].shift(-1)
unified['target_trucks_24h'] = unified['target_import_24h'] * 0.6 / 2  # 60% rail, 2 TEU/truck

print("✅ Targets created")

Creating prediction targets...
✅ Targets created


In [37]:
# Preview unified dataset
print("\n📊 UNIFIED FREIGHT MODEL SCHEMA:")
print(f"Columns: {len(unified.columns)}")
print("\nKey columns:")
key_cols = ['portname', 'date', 'portcalls', 'import_container',
            'terminal_state', 'distance_km', 'drayage_time_min',
            'surge_2std', 'ma7', 'zscore',
            'target_calls_24h', 'target_trucks_24h']
print(unified[key_cols].head(10).to_string())


📊 UNIFIED FREIGHT MODEL SCHEMA:
Columns: 63

Key columns:
    portname       date  portcalls  import_container terminal_state  distance_km  drayage_time_min  surge_2std       ma7    zscore  target_calls_24h  target_trucks_24h
0  Baltimore 2019-01-01          4      17005.253323             MD     0.446699         60.536038           0  4.000000  0.000000               2.0           0.000000
1  Baltimore 2019-01-02          2          0.000000             MD     0.446699         60.536038           0  3.000000 -0.707107               1.0           0.000000
2  Baltimore 2019-01-03          1          0.000000             MD     0.446699         60.536038           0  2.333333 -0.872872               6.0         744.821717
3  Baltimore 2019-01-04          6       2482.739057             MD     0.446699         60.536038           0  3.250000  1.240216               6.0        3885.762295
4  Baltimore 2019-01-05          6      12952.540984             MD     0.446699         60.536038   

---
## 7. 🤖 XGBoost Forecasting Model

In [38]:
# Try GPU XGBoost
try:
    import xgboost as xgb
    XGB_AVAILABLE = True
    print(f"✅ XGBoost: {xgb.__version__}")
except ImportError:
    XGB_AVAILABLE = False
    print("⚠️ XGBoost not available")

✅ XGBoost: 3.1.2


In [39]:
if XGB_AVAILABLE:
    # Prepare data
    print("Preparing ML data...")

    # Clean data
    ml_df = unified.dropna(subset=['target_calls_24h']).copy()

    # Select features
    feature_cols = ['portcalls', 'portcalls_container', 'portcalls_tanker',
                    'import_container', 'export_container', 'import_cargo', 'export_cargo',
                    'ma7', 'ma30', 'std7', 'zscore',
                    'day_of_week', 'month', 'is_weekend',
                    'month_sin', 'month_cos',
                    'calls_lag1', 'calls_lag3', 'calls_lag7',
                    'import_lag1', 'import_lag3', 'import_lag7',
                    'momentum_3d', 'momentum_7d',
                    'distance_km', 'drayage_time_min']

    # Filter to available columns
    feature_cols = [c for c in feature_cols if c in ml_df.columns]

    X = ml_df[feature_cols].fillna(0).replace([np.inf, -np.inf], 0)
    y_24h = ml_df['target_calls_24h'].fillna(0)
    y_72h = ml_df['target_calls_72h'].fillna(0)

    print(f"Features: {len(feature_cols)}")
    print(f"Samples: {len(X):,}")

Preparing ML data...
Features: 26
Samples: 20,184


In [40]:
if XGB_AVAILABLE:
    # Time-based split
    split_idx = int(len(X) * 0.8)

    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y_24h.iloc[:split_idx], y_24h.iloc[split_idx:]

    print(f"Train: {len(X_train):,}, Test: {len(X_test):,}")

    # Create DMatrix
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

Train: 16,147, Test: 4,037


In [41]:
if XGB_AVAILABLE:
    # Train XGBoost with GPU
    print("\nTraining XGBoost (GPU)...")
    t0 = time.time()

    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'max_depth': 8,
        'learning_rate': 0.1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'tree_method': 'hist',  # Use 'gpu_hist' if GPU available
        'device': 'cuda' if GPU else 'cpu',
        'seed': 42
    }

    # Try GPU
    try:
        params['tree_method'] = 'gpu_hist'
        model = xgb.train(params, dtrain, num_boost_round=100,
                         evals=[(dtest, 'test')], verbose_eval=20)
    except:
        params['tree_method'] = 'hist'
        params['device'] = 'cpu'
        model = xgb.train(params, dtrain, num_boost_round=100,
                         evals=[(dtest, 'test')], verbose_eval=20)

    print(f"\n⏱️ Training time: {time.time()-t0:.1f}s")


Training XGBoost (GPU)...
[0]	test-rmse:3.92344
[20]	test-rmse:1.58294
[40]	test-rmse:1.42037
[60]	test-rmse:1.40575
[80]	test-rmse:1.40329
[99]	test-rmse:1.40613

⏱️ Training time: 33.7s


In [42]:
if XGB_AVAILABLE:
    # Evaluate
    from sklearn.metrics import mean_absolute_error, r2_score

    y_pred = model.predict(dtest)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print("\n" + "="*60)
    print("🎯 XGBOOST 24H FORECAST PERFORMANCE")
    print("="*60)
    print(f"MAE: {mae:.2f} port calls")
    print(f"R²: {r2:.3f} ({r2*100:.1f}% variance explained)")


🎯 XGBOOST 24H FORECAST PERFORMANCE
MAE: 1.16 port calls
R²: -0.053 (-5.3% variance explained)


In [43]:
if XGB_AVAILABLE:
    # Feature importance
    importance = model.get_score(importance_type='gain')
    feat_imp = pd.DataFrame({
        'feature': list(importance.keys()),
        'importance': list(importance.values())
    }).sort_values('importance', ascending=False)

    print("\n📊 TOP 15 FEATURES:")
    print(feat_imp.head(15).to_string(index=False))


📊 TOP 15 FEATURES:
     feature  importance
         ma7  675.491821
        ma30  603.693726
 distance_km  588.557983
  calls_lag7  195.717926
  calls_lag3   41.968151
 day_of_week   37.339241
       month   29.741972
 import_lag3   29.027304
 momentum_3d   27.031610
export_cargo   26.141361
 import_lag7   25.518400
        std7   25.311661
   portcalls   25.015192
 momentum_7d   24.783566
   month_sin   24.718473


---
## 8. 💾 Save Results

In [44]:
# Save unified dataset
print("Saving results...")

# Save unified freight model
unified.to_parquet(OUTPUT_DIR / "unified_freight_model.parquet", index=False)
print(f"✅ unified_freight_model.parquet ({len(unified):,} records)")

# Save port-terminal mapping
port_terminal_df.to_csv(OUTPUT_DIR / "port_terminal_mapping.csv", index=False)
print(f"✅ port_terminal_mapping.csv ({len(port_terminal_df)} ports)")

# Save surge analysis
surge_summary.to_csv(OUTPUT_DIR / "surge_analysis_major_ports.csv", index=False)
print(f"✅ surge_analysis_major_ports.csv ({len(surge_summary)} ports)")

# Save model results
if XGB_AVAILABLE:
    results = {
        'total_ports': int(unified['portname'].nunique()),
        'total_records': len(unified),
        'features': len(feature_cols),
        'xgboost_24h': {'mae': float(mae), 'r2': float(r2)},
        'ports_with_terminals': len(port_terminal_df),
        'avg_drayage_km': float(port_terminal_df['distance_km'].mean()),
        'total_daily_trucks': float(port_terminal_df['daily_trucks_needed'].sum())
    }

    with open(OUTPUT_DIR / "pipeline_results.json", 'w') as f:
        json.dump(results, f, indent=2)
    print(f"✅ pipeline_results.json")

    # Save feature importance
    feat_imp.to_csv(OUTPUT_DIR / "xgboost_feature_importance.csv", index=False)
    print(f"✅ xgboost_feature_importance.csv")

Saving results...
✅ unified_freight_model.parquet (20,192 records)
✅ port_terminal_mapping.csv (105 ports)
✅ surge_analysis_major_ports.csv (8 ports)
✅ pipeline_results.json
✅ xgboost_feature_importance.csv


In [45]:
# FINAL SUMMARY
print("\n" + "="*80)
print("🎉 PORT-TO-RAIL PIPELINE COMPLETE!")
print("="*80)

print(f"\n📊 DATA INTEGRATION:")
print(f"   • US Ports processed: {unified['portname'].nunique()}")
print(f"   • Ports mapped to terminals: {len(port_terminal_df)}")
print(f"   • Total records: {len(unified):,}")
print(f"   • Avg port-to-terminal distance: {port_terminal_df['distance_km'].mean():.1f} km")

print(f"\n🚚 LOGISTICS ESTIMATES:")
print(f"   • Avg drayage time: {port_terminal_df['drayage_time_min'].mean():.0f} min")
print(f"   • Avg drayage cost: ${port_terminal_df['drayage_cost_usd'].mean():.0f}")
print(f"   • Total daily trucks needed: {port_terminal_df['daily_trucks_needed'].sum():.0f}")

print(f"\n🚨 SURGE ANALYSIS (Major Ports):")
print(f"   • Ports analyzed: {len(surge_summary)}")
print(f"   • Avg surge rate: {surge_summary['surge_rate'].mean():.2f}%")

if XGB_AVAILABLE:
    print(f"\n🤖 MODEL PERFORMANCE:")
    print(f"   • XGBoost 24h: MAE={mae:.2f}, R²={r2:.3f}")

print(f"\n✅ GPU Accelerated: {GPU}")


🎉 PORT-TO-RAIL PIPELINE COMPLETE!

📊 DATA INTEGRATION:
   • US Ports processed: 8
   • Ports mapped to terminals: 105
   • Total records: 20,192
   • Avg port-to-terminal distance: 2.8 km

🚚 LOGISTICS ESTIMATES:
   • Avg drayage time: 63 min
   • Avg drayage cost: $157
   • Total daily trucks needed: 57327

🚨 SURGE ANALYSIS (Major Ports):
   • Ports analyzed: 8
   • Avg surge rate: 0.64%

🤖 MODEL PERFORMANCE:
   • XGBoost 24h: MAE=1.16, R²=-0.053

✅ GPU Accelerated: True


---
## 📋 Unified Dataset Schema

```
Port Data:
- portid, portname, port_lat, port_lon
- date, year, month, day
- portcalls, portcalls_container, portcalls_tanker...
- import_container, export_container, import_cargo, export_cargo

Terminal Mapping:
- terminal_id, terminal_state, terminal_lat, terminal_lon
- distance_km (port to terminal)

Drayage Estimates:
- drayage_time_min, drayage_time_p25/p50/p75
- drayage_cost_usd
- daily_trucks_needed

Surge Indicators:
- ma7, ma30, std7
- zscore, surge_2std, surge_relative

Temporal Features:
- day_of_week, is_weekend, month_sin, month_cos
- calls_lag1/3/7, import_lag1/3/7
- momentum_3d, momentum_7d

Prediction Targets:
- target_calls_24h, target_calls_72h
- target_surge_24h
- target_trucks_24h
```